# RAG Manual de Tránsito

In [13]:
import os
from dotenv import load_dotenv
from pathlib import Path

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from langchain.load import dumps, loads

from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

import re

In [2]:
# Define the path to the .env file (one directory above)
dotenv_path = '.env'

# Load the .env file
load_dotenv(dotenv_path)

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')
langsmith_api_key = os.getenv('LANGSMITH_API_KEY')

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = langsmith_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

In [4]:
#### INDEXING ####

# Load local text file
file_path = "ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt"
loader = TextLoader(file_path)
docs = loader.load()

# Split the text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(docs)

# Index
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [12]:
splits

[Document(metadata={'source': 'ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt'}, page_content='LEY 769 DE 2002\n(Agosto 6)\n"Por la cual se expide el Código Nacional de Tránsito Terrestre y se dictan otras disposiciones".\nPODER PÚBLICO - RAMA LEGISLATIVA\nVer la Exposición de Motivos, Ley 769 de 2002\nTITULOI\nDISPOSICIONES GENERALES\nCAPITULO I\nPrincipios\nARTÍCULO 1°. ÁMBITO DE APLICACIÓN Y PRINCIPIOS. Las normas del presente Código rigen en todo el territorio nacional y regulan la circulación de los peatones, usuarios, pasajeros, conductores, motociclistas, ciclistas, agentes de tránsito, y vehículos por las vías públicas o privadas que están abiertas al público, o en las vías privadas, que internamente circulen vehículos; así como la actuación y procedimientos de las autoridades de tránsito.'),
 Document(metadata={'source': 'ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt'}, page_content='En desarrollo de lo dispuesto por el artículo 24 de la Co

In [5]:
len(splits)

275

In [6]:
vectorstore

### Prompt

In [7]:
# Multi Query: Different Perspectives
template = """Eres una IA asistente modelo de lenguaje. Tu tarea es generar cinco
diferentes versiones de la pregunta dada por el usuario para extraer los documentos
relevantes de una base de datos de vectores. Al generar múltiples perspectivas
de la pregunta del usuario, tu objetivo es ayudar al usuario a superar algunas
de las limitaciones de la búsqueda de similaridad basada en distancia. Escribe
estas preguntas alternativas separadas por caracteres de nueva línea, sin enumerar
ni listar. Pregunta original: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [8]:
generate_queries.invoke("una moto puede ir por la linea punteada amarilla entre carros")

['¿Es posible que una motocicleta circule por la línea punteada amarilla entre los automóviles?',
 '¿Puede una moto transitar por la línea punteada amarilla entre los vehículos?',
 '¿Está permitido que una motocicleta se desplace por la línea punteada amarilla entre los coches?',
 '¿Se permite que una moto vaya por la línea punteada amarilla entre los carros?',
 '¿Una motocicleta puede circular por la línea punteada amarilla entre los automóviles?']

In [9]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    print("Total number of docs retrieved", len(flattened_docs))
    unique_docs = list(set(flattened_docs))
    print("Total number of unique docs retrieved", len(unique_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "una moto puede ir por la linea amarilla entre carros"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})

Total number of docs retrieved 20
Total number of unique docs retrieved 10


C:\Users\Camilo\AppData\Local\Temp\ipykernel_26252\2937488793.py:11: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


In [10]:
docs

[Document(metadata={'source': 'ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt'}, page_content='El tránsito de actividades colectivas en vías públicas, será regulado por la autoridad local competente, teniendo en cuenta el señalamiento de velocidades y la utilización de vías para que no afecten la normal circulación de los vehículos. De igual manera, la autoridad regulará el tránsito durante la ocurrencia de otras actividades multitudinarias que impliquen la utilización de las vías destinadas a los vehículos.'),
 Document(metadata={'source': 'ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt'}, page_content='CONDUCCIÓN DE VEHÍCULOS.\nARTÍCULO 60. OBLIGATORIEDAD DE TRANSITAR POR LOS CARRILES\nDEMARCADOS. Los vehículos deben transitar, obligatoriamente, por sus respectivos carriles, dentro de las líneas de demarcación, y atravesarlos solamente para efectuar maniobras de adelantamiento o de cruce.\nPARÁGRAFO 1o. Los conductores no podrán transitar con vehíc

In [ ]:
# RAG
template = """Responde la siguiente pregunta basado en este contexto:

{context}

Pregunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

20


'Según el contexto proporcionado de la Ley 769 de 2002, Código Nacional de Tránsito de Colombia, no se menciona específicamente si una motocicleta puede transitar por la línea amarilla entre carros. Sin embargo, el artículo 60 establece que los vehículos deben transitar por sus respectivos carriles y solo pueden atravesarlos para efectuar maniobras de adelantamiento o cruce, siempre anunciando la intención con luces direccionales y de manera que no entorpezca el tránsito ni ponga en peligro a otros vehículos o peatones.\n\nPor lo tanto, transitar por la línea amarilla entre carros podría interpretarse como una infracción a estas normas, ya que no se estaría utilizando el carril de manera adecuada y podría poner en riesgo la seguridad vial. Es recomendable seguir las normas de tránsito establecidas y utilizar los carriles de manera correcta.'

### Another approach to reading metadata

In [14]:
# Load local text file
file_path = "ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt"
loader = TextLoader(file_path)
docs = loader.load()

def extract_article_sections(text):
    """
    Extract sections of text between article markers and assign article numbers.
    Returns a list of (text, article_number) tuples.
    """
    # Split the text by "ARTICULO" markers
    article_pattern = r'ARTÍCULO\s+(\d+)'
    
    # Find all article markers with their positions
    article_matches = list(re.finditer(article_pattern, text))
    
    article_sections = []
    
    # Process each article section
    for i in range(len(article_matches)):
        start_pos = article_matches[i].start()
        article_num = article_matches[i].group(1)
        
        # Get end position (either next article or end of text)
        if i < len(article_matches) - 1:
            end_pos = article_matches[i + 1].start()
        else:
            end_pos = len(text)
        
        section_text = text[start_pos:end_pos]
        article_sections.append((section_text, article_num))
    
    return article_sections

# Process the original text to get article sections
original_text = docs[0].page_content
article_sections = extract_article_sections(original_text)

# Modified text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

# Create splits with article metadata
splits = []
for section_text, article_num in article_sections:
    section_splits = text_splitter.create_documents(
        texts=[section_text],
        metadatas=[{
            "source": file_path,
            "source_article": article_num
        }]
    )
    splits.extend(section_splits)

# Index with the enhanced metadata
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [16]:
article_sections

[('ARTÍCULO 1°. ÁMBITO DE APLICACIÓN Y PRINCIPIOS. Las normas del presente Código rigen en todo el territorio nacional y regulan la circulación de los peatones, usuarios, pasajeros, conductores, motociclistas, ciclistas, agentes de tránsito, y vehículos por las vías públicas o privadas que están abiertas al público, o en las vías privadas, que internamente circulen vehículos; así como la actuación y procedimientos de las autoridades de tránsito.\nEn desarrollo de lo dispuesto por el artículo 24 de la Constitución Política, todo colombiano tiene derecho a circular libremente por el territorio nacional, pero está sujeto a la intervención y reglamentación de las autoridades para garantía de la seguridad y comodidad de los habitantes, especialmente de los peatones y de los discapacitados físicos y mentales, para la preservación de un ambiente sano y la protección del uso común del espacio público.\nLe corresponde al Ministerio de Transporte como autoridad suprema de tránsito definir, orien

In [15]:
splits

[Document(metadata={'source': 'ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt', 'source_article': '1'}, page_content='ARTÍCULO 1°. ÁMBITO DE APLICACIÓN Y PRINCIPIOS. Las normas del presente Código rigen en todo el territorio nacional y regulan la circulación de los peatones, usuarios, pasajeros, conductores, motociclistas, ciclistas, agentes de tránsito, y vehículos por las vías públicas o privadas que están abiertas al público, o en las vías privadas, que internamente circulen vehículos; así como la actuación y procedimientos de las autoridades de tránsito.'),
 Document(metadata={'source': 'ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt', 'source_article': '1'}, page_content='En desarrollo de lo dispuesto por el artículo 24 de la Constitución Política, todo colombiano tiene derecho a circular libremente por el territorio nacional, pero está sujeto a la intervención y reglamentación de las autoridades para garantía de la seguridad y comodidad de los 